In [2]:
# Generate the GRCh38 positions and HGVS protein IDs for variants in the data/SC_TD_D20_F1_training_dataset1.csv file. 
# The output is written to the data/SC_TD_D20_F1_training_dataset1_GRCh38.csv file.
import pandas as pd
# import hgvs
import os
# import liftover


In [3]:
os.chdir('/mnt/data/hong/courses/varianceAnalysis/')

In [18]:
data = pd.read_csv('data/SC_TD_D20_F1_training_dataset1.csv', sep='\t')

## generate vcf file

In [7]:
vcf_df = pd.DataFrame(columns=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE'])
vcf_df['CHROM'] = data['chromosome']
vcf_df['POS'] = data['position']
vcf_df['REF'] = data['reference']
vcf_df['ALT'] = data['alternative']
vcf_df['ID'] = '.'
vcf_df['QUAL'] = '.'
vcf_df['FILTER'] = '.'
vcf_df['INFO'] = '.'
vcf_df['FORMAT'] = '.'
vcf_df['SAMPLE'] = '.'
## write the vcf file
vcf_df.to_csv('data/SC_TD_D20_F1_training_dataset1.vcf', sep='\t', index=False)

## Use VEP web tool

## liftover genome position
make a bed file and input to ucsc liftover web tool

In [11]:
# from liftover import get_lifter
from liftover import ChainFile
def covector_row(row):
    converter = ChainFile('data/hg19ToHg38.over.chain.gz', 'hg18', 'hg38')
    chrom = row['chromosome']
    pos = row['position']
    return converter.convert_coordinate(chrom, pos)
hg38 = data.apply(covector_row, axis=1)

KeyboardInterrupt: 

In [ ]:
## make a bed file from data
bed_df = pd.DataFrame(columns=['CHROM', 'START', 'END', 'NAME'])
bed_df['CHROM'] = 'chr'+ data['chromosome']
bed_df['START'] = data['position']
bed_df['END'] = data['position'] + 1
bed_df['NAME'] = data['chromosome'] + ':' + data['position'].astype(str)
bed_df.to_csv('data/SC_TD_D20_F1_training_dataset1.bed', sep='\t', index=False)

## merge the output bed file with the original data

In [19]:
# add a column with the same "NAME" as the bed file
data['NAME'] = data['chromosome'] + ':' + data['position'].astype(str)
hg38 = pd.read_csv('data/hglft_genome_SC_TD_D20_F1_training_dataset1.bed', sep='\t')
hg38.columns=['CHROM', 'START', 'END', 'NAME', 'LEN']
## merge the bed file with the original data
data_hg38 = pd.merge(data, hg38, on='NAME', how='left')
## read in VEP file
vep = pd.read_csv('data/SC_TD_D20_F1_training_dataset1.vep', sep='\t')
# merge the VEP file with the original data
# split the "Location" column into "NAME" and "End"
vep['NAME'] = vep['Location'].str.split('-').str[0]
data_hg38_vep = pd.merge(data_hg38, vep, on = 'NAME', how='left')

In [27]:
data_clean = data_hg38_vep[['chromosome', 'position', 'reference', 'alternative', 'category', 'CHROM', 'START', 'Extra']]
## regex to extract the HGVS protein ID
## which is following HGVSp= and before ;
data_clean['HGVS'] = data_clean['Extra'].str.extract(r'HGVSp=(.*?);')
# rename the columns
data_clean.columns = ['chromosome', 'position', 'reference', 'alternative', 'category', 'hg38_chromosome', 'hg38_position', 'Extra', 'HGVSp']
# drop the "Extra" column
data_clean = data_clean.drop('Extra', axis=1)
## write the output
data_clean.to_csv('data/SC_TD_D20_F1_training_dataset1_GRCh38_HGVSp.tsv', index=False, sep='\t')